<a href="https://colab.research.google.com/github/esintao/Konrad-og-Esin/blob/main/Chain_of_thought.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from datasets import load_dataset
from transformers import pipeline

# Load model

In [67]:
prompt = "Write an email about an alpaca that likes flan"

model = pipeline(task="text2text-generation", model="google/flan-t5-base")

model(prompt, max_length=128, do_sample=True)

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "Alpaca: I have an alpaca that likes flan. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he'"}]

# Load dataset

In [5]:
import pandas as pd

In [55]:
dataset = load_dataset("derek-thomas/ScienceQA")

df = pd.DataFrame.from_dict(dataset['train'])

df = df[(df['choices'].str.len() == 4) & (df['image'].isna())]

df = df.sample(n=200)



# Prompting without CoT

In [35]:
answers = []
question_list = []

for i, row in df.iterrows():
    question_text = (
        f"Question: {row['question']}\n"
        f"(A) {row['choices'][0]} "
        f"(B) {row['choices'][1]} "
        f"(C) {row['choices'][2]} "
        f"(D) {row['choices'][3]}"
        f"The answer must be formatted as 1 letter, A-D."
    )

    answers.append(model(question_text, max_new_tokens=1, do_sample=False))
    question_list.append(question_text)

answers[0]

[{'generated_text': 'A'}]

In [36]:
question_list[0]

'Question: How long is a rowboat?\n(A) 3 centimeters (B) 3 kilometers (C) 3 meters (D) 3 millimetersThe answer must be formatted as 1 letter, A-D.'

# Prompting with CoT

In [ ]:
answers_CoT = []
question_list_CoT = []

df = df.sample(n=1)

for i, row in df.iterrows():
    # Constructing a clearly formatted prompt
    question_text_CoT = (
        "You are a careful reasoning assistant that answers multiple-choice questions about science. "
        "First, think step by step. Then, give the final answer. "
        "Format exactly as:\n"
        "Reasoning: <step 1>\n"
        "Reasoning: <step 2>\n"
        "Reasoning: <step 3>\n"
        "Answer: <final letter>\n\n"
        f"Question: {row['question']}\n"
        f"(A) {row['choices'][0]}\n"
        f"(B) {row['choices'][1]}\n"
        f"(C) {row['choices'][2]}\n"
        f"(D) {row['choices'][3]}\n\n"
        "Give three reasoning steps and at last the answer:" # This nudge helps the model start correctly
    )

    # Use a higher max_new_tokens to allow for reasoning steps
    output = model(question_text_CoT, max_new_tokens=300, do_sample=False)
    answers_CoT.append(output)
    question_list_CoT.append(question_text_CoT)

answers_CoT[0]



In [ ]:
question_list_CoT[0]